Fine tune the transformer model pretrained as below to understand sequence labels and predict a label there after.

https://huggingface.co/bert-base-uncased

Tutorial : https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt

In [1]:
! pip install -U --quiet accelerate
! pip install -U --quiet transformers
! pip install -U --quiet evaluate
! pip install -U --quiet  datasets


## Imports

In [2]:
import torch
from tabulate import tabulate
from transformers import AutoTokenizer, AdamW, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import evaluate
from datasets import load_dataset

## Load the base model

In [3]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Load the data set to be used for finetuning
The MRPC dataset contains 5,801 pairs of sentences, with a label indicating if they are paraphrases or not.

In [4]:
raw_datasets = load_dataset('glue', 'mrpc')
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [5]:
raw_train_dataset = raw_datasets["train"]
print(tabulate(zip(raw_train_dataset[10:15]['sentence1'], raw_train_dataset[10:15]['sentence2'], raw_train_dataset[10:15]['label']), headers=["Sentence 1", "Sentence 2", "Label"]))

Sentence 1                                                                                                                               Sentence 2                                                                                                                                                        Label
---------------------------------------------------------------------------------------------------------------------------------------  --------------------------------------------------------------------------------------------------------------------------------------------------------------  -------
Legislation making it harder for consumers to erase their debts in bankruptcy court won overwhelming House approval in March .           Legislation making it harder for consumers to erase their debts in bankruptcy court won speedy , House approval in March and was endorsed by the White House .        0
The Nasdaq composite index increased 10.73 , or 0.7 percent , to 1,514.77 .          

### Tokenize the dataset.

In [6]:
def tokenize_sentences(row):
    return tokenizer(row['sentence1'], row['sentence2'], truncation=True)

tokenized_dataset = raw_datasets.map(tokenize_sentences, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_dataset_padding = data_collator({k: v for k,v in tokenized_dataset["train"][:-1].items() if k not in ["sentence1", "sentence2", "idx"]})

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
  logging_steps=500,
)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()


Step,Training Loss
500,0.565000
1000,0.413300
1500,0.265500
2000,0.133300
2500,0.059200
3000,0.038100
3500,0.016800
4000,0.016900


### Prediction from the model

In [12]:
predictions = trainer.predict(tokenized_dataset["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


In [13]:
preds = np.argmax(predictions.predictions, axis=-1)


In [14]:
print(tabulate(zip(predictions.predictions[:10], preds[:10]), headers=["Predictions (logits)", "Argmax"]))

Predictions (logits)       Argmax
-----------------------  --------
[-5.355558  4.613573]           1
[ 4.89467   -4.2336354]         0
[-5.1317     4.3466177]         1
[-5.364486  4.583232]           1
[ 4.908994 -4.222063]           0
[-5.2988286  4.49859  ]         1
[-5.2689104  4.461508 ]         1
[-5.376944  4.616479]           1
[-5.359364  4.554792]           1
[-5.3868475  4.619655 ]         1


/usr/local/lib/python3.10/dist-packages/tabulate/__init__.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (len(row) >= 1 and row[0] == SEPARATING_LINE)


### Evaluate the predictions

In [15]:
metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8431372549019608, 'f1': 0.891156462585034}